In [0]:
import torch
torch.cuda.empty_cache()

In [2]:
with torch.autograd.detect_anomaly():

  import numpy as np
  import torch
  import torch.nn as nn
  import torch.optim as optim
  from torch.autograd import Variable
  import torch.utils as utils
  import torchvision.datasets as dset
  import torchvision.transforms as transforms
  import matplotlib.pyplot as plt
  from torchsummary import summary

  device = torch.device('cuda')

  def pose_loss(output_1,output_2, target_1, target_2):
      with torch.no_grad():
        P = torch.dot(output_1, output_2)
        P_truth = torch.dot(target_1, target_2)
        loss = (P - P_truth)**2
        return loss 

  def now_loss(output_1,target_1):
      return torch.mean((output_1 - target_1) ** 2)

  def my_loss(out, tar):
      loss = 0
      loss = loss + pose_loss(out[0],out[1],tar[0],tar[1])
      loss = loss + pose_loss(out[1],out[2],tar[1],tar[2])
      loss = loss + pose_loss(out[2],out[3],tar[2],tar[3])
      loss = loss + pose_loss(out[3],out[4],tar[3],tar[4])
      loss = loss + pose_loss(out[0],out[2],tar[0],tar[2])
      loss = loss + pose_loss(out[2],out[4],tar[2],tar[4])
      loss = loss + pose_loss(out[0],out[4],tar[0],tar[4]) 
      loss = loss + now_loss(out[0],tar[0])
      loss = loss + now_loss(out[1],tar[1])
      loss = loss + now_loss(out[2],tar[2])
      loss = loss + now_loss(out[3],tar[3])
      loss = loss + now_loss(out[4],tar[4])
      return loss

  class Tobi_model(nn.Module):
      def __init__(self):
          super(Tobi_model, self).__init__()
          self.Res = ResNet50()
          self.Res_5 = res_5()
          self.Res_5_2 = res_5_2()
          self.ELU = nn.ELU()
          self.rnn = nn.LSTM(input_size=int(98304/24), hidden_size=1000, num_layers=2, batch_first=True)
          self.rnn_2 = nn.LSTM(input_size=int(98304/24), hidden_size=1000, num_layers=1, batch_first=True)
          self.fc1 = nn.Linear(1000, 1024)
          self.fc2 = nn.Linear(2048, 1024)
          self.fc3 = nn.Linear(1024, 1024)
          self.final_1 = nn.Linear(1024, 3)
          self.final_2 = nn.Linear(1024, 4)

      def forward(self, x):
          x_1 = x[0].unsqueeze(0).clone()  # t-1  step
          x_2 = x[1].unsqueeze(0).clone() # t step
          
          #Residual block pass 
          x_1 = self.Res(x_1) 
          x_2 = self.Res(x_2)

          #RCNN1
          x_3 = torch.cat([x_1,x_2],dim = 1)
          x_3 = self.Res_5_2(x_3)
          # x_3 = nn.functional.avg_pool2d(x_3, 4)
          x_3, h_c = self.rnn(x_3)
          x_3 = self.fc1(x_3)
          #RCNN2
          x_2 = self.Res_5(x_2)
          # x_2 = nn.functional.avg_pool2d(x_2, 4)

          x_2, h_c_2 = self.rnn_2(x_2)
          x_2 = self.fc1(x_2)
          x_2 = self.ELU(x_2)
          #FC layer
          x_2 = x_2.view(1,1024)
          x_3 = x_3.view(1,1024)
          x_3 = torch.cat([x_2,x_3],dim = 1)
          # print(x_2.size())
          # print(x_3.size())
          x_3 = self.fc2(x_3)

          #Translation
          x_4 = self.fc3(x_3)
          x_4 = self.final_1(x_4)

          #Quanternion
          x_5 = self.fc3(x_3)
          x_5 = self.final_2(x_3)
          out = torch.cat([x_4,x_5], dim = 1)
          return out

      # def forward(self, x)


  ## ResNet Block
  class ResNet(nn.Module):

      def __init__(self, block, num_blocks):
          super(ResNet, self).__init__()
          self.in_planes = 64 
          self.conv1 = nn.Conv2d(3, 64, kernel_size=3,stride=1, padding=1, bias=False)
          self.bn1 = nn.BatchNorm2d(64)
          self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
          self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
          self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
          self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
          self.linear = nn.Linear(2048, 1024)
          self.ELU = nn.ELU(inplace=False)


      def _make_layer(self, block, planes, num_blocks, stride):
          strides = [stride] + [1]*(num_blocks-1)
          layers = []
          for stride in strides:
              layers.append(block(self.in_planes, planes, stride))
              self.in_planes = planes * block.expansion
          return nn.Sequential(*layers)


      def forward(self, x): # 4번째 layer 및 ave_pool은 하지 않음
          out = self.conv1(x) 
          out = self.bn1(out)
          out = self.ELU(out)
          out = self.layer1(out)
          out = self.layer2(out)
          out = self.layer3(out)
  #         out = self.layer4(out)  
  #         out = nn.functional.avg_pool2d(out, 4)
          return out


  #Bottleneck 구조
  class Bottleneck(nn.Module):
      expansion = 4
      def __init__(self, in_planes, planes, stride=1):
          super(Bottleneck, self).__init__()
          self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
          self.bn1 = nn.BatchNorm2d(planes)
          self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                                stride=stride, padding=1, bias=False)
          self.bn2 = nn.BatchNorm2d(planes)
          self.conv3 = nn.Conv2d(planes, self.expansion *
                                planes, kernel_size=1, bias=False)
          self.bn3 = nn.BatchNorm2d(self.expansion*planes)

          self.shortcut = nn.Sequential()
          if stride != 1 or in_planes != self.expansion*planes:
              self.shortcut = nn.Sequential(
                  nn.Conv2d(in_planes, self.expansion*planes,
                            kernel_size=1, stride=stride, bias=False),
                  nn.BatchNorm2d(self.expansion*planes)
              )
          self.elu = nn.ELU(inplace = True) # ELU 추가됨
      #Bottle Neck 한 덩어리 이게 3,4,6,3번 반복되면 Resnet 50!

      def forward(self, x):
          out = self.elu(self.bn1(self.conv1(x)))
          out = self.elu(self.bn2(self.conv2(out)))
          out = self.bn3(self.conv3(out))
          out = out + self.shortcut(x)
          out = self.elu(out)
          return out




  # 5번째 ResNet 블럭
  class ResNet_5(nn.Module):
      def __init__(self, block):
          super(ResNet_5, self).__init__()
          self.in_planes = 1024
          self.conv1 = nn.Conv2d(512, 64, kernel_size=3, stride=1, padding=1, bias=False)
          self.bn1 = nn.BatchNorm2d(64)        
          self.layer1_ = self._make_layer(block, 512, 3, stride=2)
          self.ELU = nn.ELU()
          self.linear = nn.Linear(2048, 1024)
          
      def _make_layer(self, block, planes, num_blocks, stride):
          strides = [stride] + [1]*(num_blocks-1)
          layers = []
          for stride in strides:
              layers.append(block(self.in_planes, planes, stride))
              self.in_planes = planes * block.expansion
          return nn.Sequential(*layers)

      # 5번쨰 Resdual BLock후, 펴주자
      def forward(self, x):
          out = self.layer1_(x)
          out = self.ELU(out)
          out = nn.functional.avg_pool2d(out, 16)
          out = out.view(1,out.size(0), -1)
          # out = self.linear(out)
          return out

  class ResNet_5_2(nn.Module):
      def __init__(self, block):
          super(ResNet_5_2, self).__init__()
          self.in_planes = 2048
          self.conv1 = nn.Conv2d(512, 64, kernel_size=3, stride=1, padding=1, bias=False)
          self.bn1 = nn.BatchNorm2d(64)        
          self.layer1_ = self._make_layer(block, 512, 3, stride=2)
          self.ELU = nn.ELU()
          self.linear = nn.Linear(2048, 1024)
          
      def _make_layer(self, block, planes, num_blocks, stride):
          strides = [stride] + [1]*(num_blocks-1)
          layers = []
          for stride in strides:
              layers.append(block(self.in_planes, planes, stride))
              self.in_planes = planes * block.expansion
          return nn.Sequential(*layers)

      # 5번쨰 Resdual BLock후, 펴주자
      def forward(self, x):
          out = self.layer1_(x)
          out = self.ELU(out)
          out = nn.functional.avg_pool2d(out, 16)
          out = out.view(1,out.size(0), -1)
          # out = self.linear(out)
          return out

  def res_5():
      return  ResNet_5(Bottleneck)

  def res_5_2():
      return  ResNet_5_2(Bottleneck)

  def ResNet50():
      return ResNet(Bottleneck, [3, 4, 6, 3])

  def test():
      net = ResNet18()
      y = net(torch.randn(1, 3, 32, 32))
      # print(y.size())


  class loss_cal(nn.Module):
      def __init__(self,label):
          super(loss_cal, self).__init__()
          self.out_tensor=torch.ones(5,7)
          self.label = label
          self.num = 0

      def add_loss(self, out_):
          for i in range(5):
              if i!=4:
                  self.out_tensor[i] = self.out_tensor[i+1].clone()
              else:
                  # print(out_.size())
                  self.out_tensor[i] = out_
          self.num = self.num + 1
      
      def calculate_loss(self, out__):
          if self.num>=5:
              self.add_loss(out__)
              loss = my_loss(self.out_tensor,self.label[self.num:self.num+5,:].clone())
              return loss
          else:
              self.add_loss(out__)
              pass


  print('before concat sample')
  ###############################
  ## test tensor concat sample ##
  ###############################
  data_size = 64
  batch_size = 1
  input = torch.rand(data_size,1,3,256,192,requires_grad=True).to(device)
  # input = input.int()

  resnet1_4 = ResNet50()
  resnet5 = res_5()

  pre_out_ = 0
  out_ = 0
  fin_out_ = 0
  im_data_stack = []
  im_data_stack_2 = []

  label = torch.ones(data_size,7)

  loss_ = loss_cal(label)

  # for i in range(int(data_size/batch_size)):
  #     if i==0:
  #         im_data_stack.append(input[i])
  #         pre_out_ = resnet1_4.forward(im_data_stack[0])
  #         pre_out_ = resnet5.forward(pre_out_)
  #     elif i==1:
  #         im_data_stack.append(input[i])
  #         out_ = resnet1_4.forward(im_data_stack[1])
  #         out_ = resnet5.forward(out_)
  #         fin_out_ = torch.cat([pre_out_,out_],dim=0)
  #     else:
  #         pre_out_ = out_
  #         del im_data_stack[0]
  #         im_data_stack.append(input[i])
  #         out_ = resnet1_4.forward(im_data_stack[1])
  #         out_ = resnet5.forward(out_)
  #         fin_out_ = torch.cat([pre_out_,out_],dim=0)
  #         print(fin_out_.size()) 

  print('before Tobi model')
  # tobiVO = Tobi_model().to('cpu')
  tobiVO = Tobi_model().to(device)
  cal_loss = 0
  torch.save(tobiVO,'./modelsize')
  # print(tobiVO.eval())
  # summary(tobiVO,input_size=(3,256,192),device='cpu')

  optimizer = optim.Adam(tobiVO.parameters(), lr=0.001)
  print('before Tobi model')
  # with torch.no_grad():
  for i in range(input.size()[0]):
      temp_loss = 0
      if i>=1:
          optimizer.zero_grad()
          input_ = torch.cat([input[i-1],input[i]],dim=0)
          finout = tobiVO.forward(input_)
          print(loss_.num)

          if(loss_.num>=5):
              # print('hi')
              temp_loss = loss_.calculate_loss(finout)
              cal_loss = cal_loss + temp_loss
              cal_loss.backward(retain_graph = True)
              optimizer.step()
              print(cal_loss)
          else:
              with torch.no_grad():
                loss_.calculate_loss(finout)
          
  print('all ran')    
  print(type(cal_loss))

/usr/local/lib/python3.6/dist-packages/torch/autograd/anomaly_mode.py:70: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  warnings.warn('Anomaly Detection has been enabled. '


before concat sample
before Tobi model


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Tobi_model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Bottleneck. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type ResNet_5. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3

before Tobi model
0
1
2
3
4
5
tensor(347.8582, grad_fn=<AddBackward0>)
6


RuntimeError: ignored